In [19]:
from utils import *

import pandas as pd

from torch.utils.data import Dataset, DataLoader


import torch
from torch import nn


def read_obj_mp(mp_pth, vocab, embedding):
    """_summary_

    Args:
        mp_pth (_type_): _description_
        vocab (_type_): _description_
        embedding (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Read map data
    mp_fp = open(mp_pth, "r", encoding="utf-8")
    mp_data = mp_fp.readlines()
    mp_data = [
        (i.replace("\n", "")).split(",") for i in mp_data
    ]
    mp_fp.close()
    
    mp_ts_label = []
    mp_ts_geometric = []
    
    for idx in range(len(mp_data)):
        
        mp_ts_label += [int(vocab.add(mp_data[idx][0]))]
        mp_ts_geometric.append([
            round(float(mp_data[idx][1]), 4),
            round(float(mp_data[idx][2]), 4),
            round(float(mp_data[idx][3]), 4),
            round(float(mp_data[idx][4]), 4),
            round(float(mp_data[idx][5]), 4),
            round(float(mp_data[idx][6]), 4),
        ])
    
    # Embed semantic label
    mp_ts_label = torch.Tensor(mp_ts_label).to(dtype=torch.int)
    mp_ts_label = embedding(mp_ts_label)
        
    # Add Gaussian noise
    mp_ts_geometric = torch.Tensor(mp_ts_geometric)

    mean = 0
    std = 0.03
    mp_noise = torch.randn(mp_ts_geometric.shape) * std + mean
    mp_ts_geometric = mp_ts_geometric + mp_noise
    
    # Adjust the map tensor length
    mp_data = torch.concat((mp_ts_label, mp_ts_geometric), dim=1)
    
    mp_len = len(mp_data)
    mp_dim = len(mp_data[0])
    
    ad_mp_len = 150
    
    mp_mask_1 = torch.ones(mp_len)
    
    if mp_len < ad_mp_len:
        ad_mp_data = torch.zeros((ad_mp_len - mp_len, mp_dim))    
        mp_data = torch.concat((mp_data, ad_mp_data))

        mp_mask_2 = torch.zeros(ad_mp_len - mp_len)    
        mp_mask = torch.concat((mp_mask_1, mp_mask_2))
    
    
    return mp_data, mp_mask


class IgibsonPanoramaWithObj(Dataset):
    
    def __init__(self, csv_pth):
        super(IgibsonPanoramaWithObj, self).__init__()
        
        self.vocab = Vocab()
        self.embedding = nn.Embedding(512, 5)
        self.data = pd.read_csv(csv_pth)
        
    def __len__(self):
        
        return len(self.data)
    
    def __getitem__(self, index):
        
        # Read ground truth
        gt_pth = self.data["gt"][index]
        gt = read_gt(gt_pth)
        gt = gt[0, ...]
        
        # Read observation
        # obs_pth = self.data["rgb"][index]
        # obs = read_obs(obs_pth)
        
        obs_pth = self.data["d"][index]
        obs = read_depth(obs_pth)
        
        # Read map
        mp_pth = self.data["mp"][index]
        mp, mp_mask = read_mp(mp_pth)
        # mp, mp_mask = read_mp_ablation(mp_pth)
        
        
        return (gt, obs, mp, mp_mask)

In [22]:
mp_pth = "e:\Workspace\Datasets\iGibson-dataset\iGibson-map\Beechwood_1_int\Beechwood_1_int.txt"
vocab = Vocab()
embedding = nn.Embedding(512, 6)

mp_ts, mp_mask = read_obj_mp(mp_pth, vocab, embedding)

print(mp_ts.shape)
print(mp_mask.shape)

print(mp_mask)

torch.Size([150, 12])
torch.Size([150])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
